# Using the PETITE utilities to generate new integrator maps
Authors: Kevin J. Kelly (Texas A&M), Nikita Blinov (U. Victoria), Ryan Plestid (Caltech), Patrick J. Fox (Fermilab), and Pedro A.N. Machado (Fermilab)

! Warning: running this can be time consuming !

* First we will generate all the Standard Model processes ('Brem', 'PairProd', 'Comp', 'Ann', 'Moller', 'Bhabha') for a range of energies.
* Second we will generate integrators for dark Bremsstrahlung for a range of dark vector masses



In [11]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [12]:
# import necessary functions
import numpy as np
import os
path = os.getcwd()
path = os.path.split(path)[0]
print(path)

import generate_integrators as genint
import find_maxes
from PETITE import physical_constants as pconst

/Users/mhostert/Repos/PETITE


# Bulk generation

In [18]:

##################################
##         SM processes         ##
##################################

# List of incoming particle energies at which to calculate integrators for each process
initial_energy_list = np.geomspace(0.0016, 5e3, 500)
# Necessary parameters for generating the integrators, note save_location should be altered as preferred
training_params = {'verbosity':False, 
                   'initial_energy_list':initial_energy_list,
                   'save_location':path + '/MH_data/',
                   'run_find_maxes':True}

# Necessary parameters for processing the integrators to determine cross sections
processing_params = {'process_targets':['graphite','lead'], 'save_location':path + '/MH_data/', 'neval': 1000, 'n_trials': 500}

# List of processes to do
processes_to_do = ['Brem', 'PairProd', 'Comp', 'Ann', 'Moller', 'Bhabha']

# Loop over processes, carrying out each step of the calculation, they can also be called in one command generate_integrators
for process in processes_to_do:
    genint.make_integrators(training_params, process, overwrite=True)
    genint.stitch_integrators(training_params['save_location'] + '/' + process + '/')
    genint.cleanup(training_params['save_location'] + "/" + process + "/")

Stitched integrator saved as /Users/mhostert/Repos/PETITE/MH_data//Brem//Brem_AdaptiveMaps.npy
Cleaning up files in /Users/mhostert/Repos/PETITE/MH_data//Brem/
Stitched integrator saved as /Users/mhostert/Repos/PETITE/MH_data//PairProd//PairProd_AdaptiveMaps.npy
Cleaning up files in /Users/mhostert/Repos/PETITE/MH_data//PairProd/
Stitched integrator saved as /Users/mhostert/Repos/PETITE/MH_data//Comp//Comp_AdaptiveMaps.npy
Cleaning up files in /Users/mhostert/Repos/PETITE/MH_data//Comp/
Stitched integrator saved as /Users/mhostert/Repos/PETITE/MH_data//Ann//Ann_AdaptiveMaps.npy
Cleaning up files in /Users/mhostert/Repos/PETITE/MH_data//Ann/
Stitched integrator saved as /Users/mhostert/Repos/PETITE/MH_data//Moller//Moller_AdaptiveMaps.npy
Cleaning up files in /Users/mhostert/Repos/PETITE/MH_data//Moller/
Stitched integrator saved as /Users/mhostert/Repos/PETITE/MH_data//Bhabha//Bhabha_AdaptiveMaps.npy
Cleaning up files in /Users/mhostert/Repos/PETITE/MH_data//Bhabha/


In [19]:
# Find maxes for each process
genint.call_find_maxes(processing_params, processes_to_do)

# Need to save xsec and invmfp interpolators as well
genint.create_xsec_interp(training_params)

List of processes:  ['Brem', 'PairProd', 'Comp', 'Ann', 'Moller', 'Bhabha']
List of target materials:  ['graphite', 'lead']
[Errno 2] No such file or directory: '/Users/mhostert/Repos/PETITE/MH_data//auxiliary/Brem/Brem_AdaptiveMaps.npy'
Tried to load adaptive maps /Users/mhostert/Repos/PETITE/MH_data//auxiliary/Brem/Brem_AdaptiveMaps.npy but could not find them.
Trying again at /Users/mhostert/Repos/PETITE/MH_data/Brem/Brem_AdaptiveMaps.npy
File being processed:  /Users/mhostert/Repos/PETITE/MH_data/Brem/Brem_AdaptiveMaps.npy
[Errno 2] No such file or directory: '/Users/mhostert/Repos/PETITE/MH_data//auxiliary/PairProd/PairProd_AdaptiveMaps.npy'
Tried to load adaptive maps /Users/mhostert/Repos/PETITE/MH_data//auxiliary/PairProd/PairProd_AdaptiveMaps.npy but could not find them.
Trying again at /Users/mhostert/Repos/PETITE/MH_data/PairProd/PairProd_AdaptiveMaps.npy
File being processed:  /Users/mhostert/Repos/PETITE/MH_data/PairProd/PairProd_AdaptiveMaps.npy
[Errno 2] No such file or 

## BSM stuff (I haven't looked here yet)

In [73]:
##################################
##         Dark processes       ##
##################################
initial_energy_list = np.geomspace(0.01, 500, 500)
# Dark vector masses in GeV
#mV_list = [0.001, 0.003, 0.010, 0.030, 0.100, 0.300, 1.00]
mV_list = [0.003, 0.030, 0.300]
save_location = path + '/coarsegrid_test/'
training_params = {'verbosity': False, 
                   'initial_energy_list': initial_energy_list,
                   'save_location': save_location,
                   'run_find_maxes':True, 
                   'mV_list': mV_list, 
                   'training_target': 'hydrogen', 
                   'mT': 200.0}
processes_to_do = ['DarkBrem', 'DarkAnn','DarkComp']

for mV in mV_list:
    for process in processes_to_do:
        if process == 'DarkBrem':
            if mV > initial_energy_list[0]:
                initial_energy_list = np.logspace(np.log10(1.01*mV), np.log10(initial_energy_list[-1]), len(initial_energy_list))
                training_params.update({'initial_energy_list':initial_energy_list})
        if process == 'DarkAnn':
            if mV**2 > 2*pconst.m_electron*(initial_energy_list[0] + pconst.m_electron):
                Ee0 = 1.01*((mV**2 - 2*pconst.m_electron**2)/(2*pconst.m_electron))
                initial_energy_list = np.logspace(np.log10(Ee0), np.log10(initial_energy_list[-1]), len(initial_energy_list))
                training_params.update({'initial_energy_list':initial_energy_list})
        training_params.update({"mV":mV})
        genint.make_integrators(training_params, process, overwrite=True)
        genint.stitch_integrators(training_params['save_location'] + process + '/mV_' + str(int(np.floor(mV*1000.))) + "MeV/")
        genint.cleanup(training_params['save_location'] + process + '/mV_' + str(int(np.floor(mV*1000.))) + "MeV/")

processes_to_do = ['DarkBrem', 'DarkAnn', 'DarkComp']
processing_params = {'process_targets':['graphite','lead'], 'save_location':save_location, 'mV_list':mV_list}
genint.call_find_maxes(processing_params, processes_to_do)

Using specified m_T = 200.0
Stitched integrator saved as /Users/mhostert/Repos/PETITE/coarsegrid_test/DarkBrem/mV_3MeV//DarkBrem_AdaptiveMaps.npy
Cleaning up files in /Users/mhostert/Repos/PETITE/coarsegrid_test/DarkBrem/mV_3MeV/
Using specified m_T = 200.0


/Users/mhostert/Repos/PETITE/src/PETITE/radiative_return.py:39: RuntimeWarning: invalid value encountered in power
  (8.0 + 3.0 * beta) * np.power(1.0 - x, beta / 2.0 - 1.0) - 4.0 * (1.0 + x)
/Users/mhostert/Repos/PETITE/src/PETITE/radiative_return.py:39: RuntimeWarning: invalid value encountered in power
  (8.0 + 3.0 * beta) * np.power(1.0 - x, beta / 2.0 - 1.0) - 4.0 * (1.0 + x)
/Users/mhostert/Repos/PETITE/src/PETITE/radiative_return.py:39: RuntimeWarning: invalid value encountered in power
  (8.0 + 3.0 * beta) * np.power(1.0 - x, beta / 2.0 - 1.0) - 4.0 * (1.0 + x)
/Users/mhostert/Repos/PETITE/src/PETITE/radiative_return.py:39: RuntimeWarning: invalid value encountered in power
  (8.0 + 3.0 * beta) * np.power(1.0 - x, beta / 2.0 - 1.0) - 4.0 * (1.0 + x)
/Users/mhostert/Repos/PETITE/src/PETITE/radiative_return.py:39: RuntimeWarning: invalid value encountered in power
  (8.0 + 3.0 * beta) * np.power(1.0 - x, beta / 2.0 - 1.0) - 4.0 * (1.0 + x)
/Users/mhostert/Repos/PETITE/src/PETITE/

Stitched integrator saved as /Users/mhostert/Repos/PETITE/coarsegrid_test/DarkAnn/mV_3MeV//DarkAnn_AdaptiveMaps.npy
Cleaning up files in /Users/mhostert/Repos/PETITE/coarsegrid_test/DarkAnn/mV_3MeV/
Using specified m_T = 200.0
Stitched integrator saved as /Users/mhostert/Repos/PETITE/coarsegrid_test/DarkComp/mV_3MeV//DarkComp_AdaptiveMaps.npy
Cleaning up files in /Users/mhostert/Repos/PETITE/coarsegrid_test/DarkComp/mV_3MeV/
Using specified m_T = 200.0


/Users/mhostert/Repos/PETITE/src/PETITE/all_processes.py:276: RuntimeWarning: invalid value encountered in sqrt
  k = np.sqrt((x * Ebeam) ** 2 - mV**2)


KeyboardInterrupt: 